# Práctica 2 - Self-Organising Maps - JUGADORES
## Preparación de entorno
#### Importar librerías de código

Álvaro Chavarri, Daniel Ojeda, Carlos Vega y Lucía Gálvez

In [3]:
# from __future__ import division

import numpy as np
from matplotlib import pyplot as plt
from matplotlib import patches as patches
import pandas as pd
from sklearn import preprocessing

%matplotlib inline

#### Dataset que se va a utilizar para el entrenamiento

In [4]:
# Código para obtener el Dataset que se va a usar en el entrenamiento
file1 = 'Jugadores_Característica.csv'
file2 = 'Jugadores_DatosPersonales.csv'
file3 = 'Jugadores_Posiciones.csv'
#leemos los datasets
data1 = pd.read_csv(file1)
data2 = pd.read_csv(file2)
data3 = pd.read_csv(file3)

# creamos la matriz por separado
#datos = np.random.randint(valor_min, valor_max, (valores_color, num_colores))

C:\Users\luga_\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (11,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Solo usaremos la tabla data1

In [5]:
print(data1)

       Unnamed: 0 Acceleration Aggression Agility Balance Ball control  \
0               0           89         63      89      63           93   
1               1           92         48      90      95           95   
2               2           94         56      96      82           95   
3               3           88         78      86      60           91   
4               4           58         29      52      35           48   
5               5           79         80      78      80           89   
6               6           57         38      60      43           42   
7               7           93         54      93      91           92   
8               8           60         60      71      69           89   
9               9           78         50      75      69           85   
10             10           75         84      79      60           84   
11             11           76         68      80      75           87   
12             12           46        

In [6]:
#Eliminamos la columna Unnamed: 0 ya que se repite y no es necesaria
datos = data1.drop(columns='Unnamed: 0')
datos

,Acceleration,Aggression,Agility,Balance,Ball control,Composure,Crossing,Curve,Dribbling,Finishing,...,Reactions,Short passing,Shot power,Sliding tackle,Sprint speed,Stamina,Standing tackle,Strength,Vision,Volleys
0,89,63,89,63,93,95,85,81,91,94,...,96,83,94,23,91,92,31,80,85,88
1,92,48,90,95,95,96,77,89,97,95,...,95,88,85,26,87,73,28,59,90,85
2,94,56,96,82,95,92,75,81,96,89,...,88,81,80,33,90,78,24,53,80,83
3,88,78,86,60,91,83,77,86,86,94,...,93,83,87,38,77,89,45,80,84,88
4,58,29,52,35,48,70,15,14,30,13,...,85,55,25,11,61,44,10,83,70,11
5,79,80,78,80,89,87,62,77,85,91,...,91,83,88,19,83,79,42,84,78,87
6,57,38,60,43,42,64,17,21,18,13,...,88,50,31,13,58,40,21,64,68,13
7,93,54,93,91,92,87,80,82,93,83,...,85,86,79,22,87,79,27,65,86,79
8,60,60,71,69,89,85,85,85,79,76,...,86,90,87,69,52,77,82,74,88,82
9,78,50,75,69,85,86,68,74,84,91,...,88,75,88,18,80,72,22,85,70,88


## SOM Setup
#### Variables definidas por el alumno

In [7]:
# Inicializa tamaño del mapa de Kohonen, número de iteraciones y learning rate
lado_mapa = 20
periodo = 10000
learning_rate = 0.02 

#### A partir de este punto solo hay cálculos. No se introducen más valores "a mano"

In [16]:
# Establece el numero de entradas del mapa y el número de datos que se van a usar para entrenar. 
# Utiliza una función que obtenga automáticamente los valores a partir del Dataset.
num_entradas = len(datos.index)

# Calcula el vecindario inicial. Debe ser la mitad del lado del mapa de Kohonen
vecindario_inicial = lado_mapa/2

# Normaliza los datos usando el algoritmo MinMax() que nos ofrece la librería sklearn
##ss = preprocessing.MinMaxScaler()
#datosN = ss.fit_transform(datos)
# Normaliza los datos si fuese necesario dividiendo cada dato por el máximo en la matriz
datosN2 = (datos-datos.min())/(datos.max()-datos.min())
    
# Crea una matriz de pesos con valores random entre 0 y 1. Usa la función random.random de la librería NumPy
m_pesos = np.random.random(((lado_mapa, lado_mapa, num_entradas)))

TypeError: unsupported operand type(s) for -: 'str' and 'str'

#### Funciones para entrenar/clasificar

##### Calcular BMU
Esta función encuentra la neurona ganadora, calculando la distancia Euclídea neurona-patrón y guardando las coordenadas en la BMU (bmu_idx)

In [ ]:
# Función para encontrar la BMU
"""
   Encuentra la BMU para un patrón de entrada.
   Entradas: (patrón_de_entrada, matriz_de_pesos, número_de_entradas)
   Salidas:  (bmu, bmu_idx) tupla donde
               bmu: vector de pesos de la neurona ganadora
               bum_idx: coordenadas de la neurona ganadora
"""
def calcular_bmu(patron_entrada, m_pesos, m):
        #inicializamos la distancia minina
    dMin = np.iinfo(np.int).max
    #recorremos la matriz de pesos
    for x in range (m_pesos.shape[0]): #para x
        for y in range (m_pesos.shape[1]): #para y
            w = m_pesos[x, y] #vector de pesos
            dEuclidea = np.sqrt(np.sum((w - patron_entrada)** 2)) #distancia Euclidea
            if dEuclidea < dMin:
                dMin = dEuclidea
                bmu_idx = np.array([x, y])
    #vector de pesos de la neurona ganadora
    bmu = m_pesos[bmu_idx[0], bmu_idx[1]]
    return (bmu, bmu_idx)

##### Variación learning rate
Esta función calcula eta para la i-ésima iteración

In [ ]:
# Función para calcular el descenso del coeficiente de aprendizaje (eta)
"""
   Calcula el Learning Rate (eta) que corresponde a la i-ésima presentación.
   Entradas: (learning_rate_inicial, iteracion, período)
   Salidas:  learning_rate para la iteración i

"""
def variacion_learning_rate(learning_rate, i, periodo):
    learning_rateV = learning_rate * (1-(i/periodo))
    return learning_rateV

##### Variación vecindario
Esta función calcula el desdenso del vecindario

In [ ]:
# Función para calcular el descenso del vecindario (v)
"""
   Calcula el vecindario  (v) que corresponde a la i-ésima presentación.
   Entradas: (vecindario_inicial, iteracion, período)
   Salidas:  lvecindario para la iteración i

"""
def variacion_vecindario(vecindario_inicial, i, periodo):
    lvecindario = 1 + vecindario_inicial * (1-(i/periodo))
    return lvecindario

##### Decay
Esta función calcula eta en función de la distancia a la BMU

In [ ]:
# Función para calcular el descenso del coeficiente de aprendizaje (eta) en función de la distancia a la BMU
"""
   Calcula la amortiguación de eta en función de la distancia en el mapa entre una neurona y la BMU.
   Entradas: (distancia_BMU, vecindario_actual)
   Salidas:  amortiguación para la iteración

"""
def decay(distancia_BMU, vecindario_actual):
    return np.exp(-distancia_BMU**2 / (2*vecindario_actual**2))

#### Funciones para dibujar la salida de la red

Esta función nos permite pintar la matriz de los valores con los colores RGB

In [ ]:
# Función para pintar una matriz de valores como colores RGB
def pintar_mapa(matriz_valores):
    fig = plt.figure()
    
    # Establece ejes
    ax = fig.add_subplot(111, aspect='equal')
    ax.set_xlim((0, matriz_pesos.shape[0]+1))
    ax.set_ylim((0, matriz_pesos.shape[1]+1))
    ax.set_title('Self-Organising Map después de %d iteraciones' % periodo)

    # Dibuja los rectángulos de color RGB
    for x in range(1, matriz_valores.shape[0] + 1):
        for y in range(1, matriz_valores.shape[1] + 1):
            ax.add_patch(patches.Rectangle((x-0.5, y-0.5), 1, 1,
                         facecolor=matriz_valores[x-1,y-1,:],
                         edgecolor='none'))
    plt.show()

## SOM Entrenamiento

Entrenamos la red con el dataset 'Jugadores_Características.csv'

In [ ]:
# Entrena la red con el dataset de entrenamiento
for i in range(periodo):
    if i % 100 == 0: #cada 100 iteraciones imprimimos el número correspondiente
        print('Iteración: ', i)
    #patrón de entrada escogido al azar y convertido al formato de m_pesos
    patron_entrada = datosN[np.random.randint(datos.shape[0])] 
    #buscamos la BMU
    bmu, bmu_idx = calcular_bmu(patron_entrada, m_pesos, num_entradas)
    #calculamos eta y el vecindario correspondientes a la iteración i
    eta = variacion_learning_rate(learning_rate, i, periodo)
    v = variacion_vecindario(vecindario_inicial, i, periodo)

    #actualizamos el vector de pesos y así nos acercamos al patrón
    for x in range(lado_mapa):
        for y in range(lado_mapa):
            w = m_pesos[x, y] #actualizamos el vector de pesos
            #calculamos la distancia euclídea para cada neurona y acercarnos a la correspondiente
            distEuclidea = np.sqrt(np.sum((np.array([x,y]) - bmu_idx)**2))
            if distEuclidea <= vecindario_inicial: #si la distancia euclidea es menor o igual que el vecindario
                #calculamos la amortigüación 
                amortiguacion = decay(distEuclidea, v)
                #actualizamos el vector de pesos de la neurona
                m_pesos[x, y] = w + eta * amortiguacion * (patron_entrada - w)

m_pesos

## SOM Clasificación

### Clasifica los patrones de entrenamiento con la matriz de pesos recién entrenada

#### Definimos e inicializamos las tres matrices
##### Mapa de Clasificación: mClasificacion 
Es una matriz con las mismas dimensiones que m_pesos y guarda en cada neurona el último patron clasificado
##### Mapa de Activación: mActivacion
Es una matriz bidimensional que guarda el número de patrones reconocido por  cada neurona
##### Mapa de Distancias: mDistancias
Es una matriz bidimensional que guarda las neuronas con activación>0

In [ ]:
mapaC = np.zeros((lado_mapa, lado_mapa))    
mClasificacion = mapaC.astype(str)#convertimos los valores en strings
mActivacion = np.zeros((lado_mapa, lado_mapa))
mDistancias = np.zeros((lado_mapa, lado_mapa))
distanciaEuclidea = 0

#### Recorremos todo el dataset de entrenamiento

##### Imprimimos las coordenadas x e y de la BMU para cada patrón de prueba

In [ ]:
for i in range (0, num_colores):
    bmu, bmu_idx = calcular_bmu(datosN[i], m_pesos, num_entradas)
    #actualizamos el mapa de clasificación, añadiendo el patrón en la neurona que lo ha reconocido
    mClasificacion[bmu_idx[0], bmu_idx[1]] += 
    
    #FALTA LO DEL MAPA DE CLASIFICACION
    
    #actualizamos el mapa de activación, +1 en las activaciones
    mActivacion[bmu_idx[0], bmu_idx[1]] =+ 1
    distanciaEuclidea = np.sqrt(np.sum((bmu - datosN[i])**2)) #calculamos la distancia euclidea
    mDistancias[bmu_idx[0], bmu_idx[1]] =+ distanciaEuclidea
    print('Coordenadas BMU:', datosN[i])
    print('X = ', bmu_idx[0])
    print('Y = ', bmu_idx[1])  

##### Calculamos los mapas de clasificación, activación, y distancias

In [ ]:
print('Mapa de Clasificación')
pintar_mapa(mClasificacion)
print('\nMapa de Activación')
print(mActivacion)

#FALTA HISTOGRAMA 3D

print('\nMapa de Distancias')
print(mDistancias)

##### Calculamos el número de clases

In [ ]:
num_clases = np.count_nonzero(mActivacion)
print('Número de clases: ', num_clases)

##### Calculamos el Error de Cuantificación y el Error Topológico del mapa

In [ ]:
#error de cuantificación
eCuantificacion = (np.sum(mDistancias))/(np.sum(mActivacion))
print('Error de Cuantificación: ', eCuantificacion)

In [ ]:
#error topológico
#calculamos la segunda bmu para el error topológico
def calcular_bmu2(patron_entrada, m_pesos, num_entradas):
    #inicializamos la distancia minina
    dMin = np.iinfo(np.int).max
    bmu_idx = np.zeros(2) #le pasamos a las bmu_idx un array de ceros (2D)
    #recorremos la matriz de pesos
    for x in range (m_pesos.shape[0]): #para x
        for y in range (m_pesos.shape[1]): #para y
            w = m_pesos[x, y] #vector de pesos
            dEuclidea = np.sqrt(np.sum((w - patron_entrada)** 2)) #distancia Euclidea
            if dEuclidea < dMin:
                bmu_idx2 = bmu_idx
                dMin = dEuclidea
                bmu_idx = np.array([x, y])
   
    return (bmu_idx, bmu_idx2)


t=0
for i in range(t, num_colores):
    bmu_idx, bmu_idx2 = calcular_bmu2(datosN[i], m_pesos, num_entradas)
    x = bmu_idx[0]
    y = bmu_idx[1]
    for a in [x-1,x,x+1]:
        for b in [y-1,y,y+1]:
            if bmu_idx2[0]== a and bmu_idx2[1] == b:
                ady = True #t(x) = 0 si la primera y segunda BMU son adyacentes
            elif a == bmu_idx[0]+1 and b == bmu_idx[1]+1 and ady != True:
                t+=1 #en cambio, t(x) = 1 en caso contrario

eTopologico = t/num_colores
            

print('Error Topológico: ', eTopologico) #MAAAAAAAAAAAAL NO PUEDE DAR CERO

## SOM Prueba

In [ ]:
# Clasifica nuevos patrones

